In [1]:
#EE19B145 Siddharth Nilesh Ambekar (ANYONE WHO MAKES MODIFICATIONS SHOULD ADD THEIR NAME HERE)

#### 1)Preprocessing Functions

In [2]:
import string
from string import punctuation
import math

import nltk
from nltk.corpus import stopwords

from nltk.corpus import wordnet

from nltk import WordNetLemmatizer

import re 

alph='abcdefghijklmnopqrstuvwxyz';
num='0123456789';
Map={alph[i]:i for i in range(len(alph))};
NumMap={num[i]:i for i in range(len(num))};
Map.update(NumMap);

In [3]:
def fullCleanup(t):  #This function is useful for cosine similarity, and substring matching
    d=t.replace('\n',' ');
    L=re.split(r'\W+',d);
    L=[i.lower() for i in L];
    bucket=[];
    stop_words=set(stopwords.words('english'))-{'won','not'};
    for i in L:
        if i in stop_words or len(i)<=2:
            continue;
        else:
            bucket+=[i];
    lemmatiser=WordNetLemmatizer();
    bucket=[lemmatiser.lemmatize(i,pos='v') for i in bucket];
    bucket=[lemmatiser.lemmatize(i,pos='n') for i in bucket];
    bucket=[lemmatiser.lemmatize(i,pos=wordnet.ADJ) for i in bucket];
    bucket=[lemmatiser.lemmatize(i,pos=wordnet.ADV) for i in bucket];
    return bucket;

In [4]:
#THIS FUNCTION BREAKS THE GIVEN DOCUMENT INTO ITS COMPONENT SENTENCES.
def SentenceBreak(doc):  
    d=doc.replace('\n',' ');
    d1=d.lower();
    n=len(d1);
    ind=0;
    b=[];
    for i in range(n):
        if d1[i]=='.' or d1[i]=='!' or d1[i]=='?':
            if ind==0:
                b+=[d1[ind:i]];
                ind=i;
            else:
                b+=[d1[ind+2:i]];
                ind=i;
    return b;            

#### Functions for Cosine Similarity

In [5]:
def Vectoriser(doc):
    L=fullCleanup(doc);
    LDict={};
    for i in L:
        LDict.setdefault(i,None);
        if LDict[i]!=None:
            LDict[i]+=1;
        else:
            LDict[i]=1;
    return LDict;

In [6]:
import numpy as np

In [7]:
def CountVectorise(doc1,doc2):
    LDict1=Vectoriser(doc1);
    LDict2=Vectoriser(doc2);
    basis={};
    for i in LDict1:
        basis.setdefault(i,0);
        basis[i]=1;
    for j in LDict2:
        basis.setdefault(j,0);
        basis[j]=1;
    D1=[];
    D2=[];
    for k in basis:
        LDict1.setdefault(k,0);
        LDict2.setdefault(k,0);
        D1+=[[k,LDict1[k]]];
        D2+=[[k,LDict2[k]]];
    return [D1,D2];

In [8]:
def CosineSimilarity(doc1,doc2):
    V=CountVectorise(doc1,doc2);
    v1=np.array([V[0][k][1] for k in range(len(V[0]))]);
    v2=np.array([V[1][k][1] for k in range(len(V[1]))]);
    norm1=(np.dot(v1,v1))**0.5;
    norm2=(np.dot(v2,v2))**0.5;
    cosine=np.dot(v1,v2)/(norm1*norm2);
    return cosine;

#### Functions for string matching

In [9]:
#THIS ALGORITHM TAKES AS INPUT THE DOCUMENT AND THE STRING YOU WANT TO SEARCH IN IT; IT ALSO TAKES A DICTIONARY
#AS INPUT. THE DICTIONARY ASSIGNS NUMERICAL VALUES TO THE ALPHABETS SO THAT STRING MATCHING CAN BE DONE
#USING HASH FUNCTIONS
#THE FUNCTION RETURNS A LIST OF INDICES OF THE FORM [start, end] WHERE START INDICATES WHERE THE FIRST
#CHARACTER OF THE PATTERN IS FOUND IN THE DOCUMENT, AND END INDICATES WHERE THE LAST CHARACTER OF THE 
#PATTERN OCCURS IN THE DOCUMENT.

def RabinKarp(s,p,M):
    n=len(s);
    m=len(p);
    ref=0;
    h=0;
    b=[];
    
    for i in range(m):
        char=p[i];
        M.setdefault(char,0);
        val=M[char];
        ref+=val*(2**(m-i-1));
    
    for j in range(m):
        char=s[j];
        M.setdefault(char,0);
        val=M[char];
        h+=val*(2**(m-j-1));
    if ref==h:
        b+=[[0,m]];
    
    for k in range(1,n-m):
        c1=s[k-1];
        c2=s[k+m-1];
        M.setdefault(c1,0);
        M.setdefault(c2,0);
        v1=M[c1];
        v2=M[c2];
        h=(2*(h-v1*(2**(m-1)))+v2);
        if h==ref:
            b+=[[k,k+m]];
  #  if b!=0:
    #    print(s,p,b)
    return b;

In [10]:
#THIS FUNCTION RETURNS THE NUMBER OF SENTENCES THAT ARE POSSIBLY THE SAME FOR THE 2 DOCUMENTS. 
#IT DOESN'T RETURN WHERE THESE SENTENCES OCCUR, SINCE THAT WAS A LITTLE HARD, BUT IT TELLS HOW MANY 
#SENTENCES ARE COMMON TO THE DOCUMENTS. AS A RESULT, IT IS EASY TO CHECK HOW MUCH OF THE DOCUMENTS WERE 
#DIRECTLY COPIED WITH MINOR CHANGES.

def MatchStrings(doc1,doc2,M):
    sdoc1=SentenceBreak(doc1);
    sdoc2=SentenceBreak(doc2);
    e='';
    l1=len(sdoc1);
    l2=len(sdoc2);
    count=0;
    s1=[e.join(fullCleanup(i)) for i in sdoc1];
    d2=e.join(fullCleanup(doc2));
    for i in s1:
        b=RabinKarp(d2,i,M);
        if len(b)!=0:
            count+=1;
    return [["Number of common sentences",count],["Number of sentences in the first document:",l1],["Number of sentences in the second document:",l2]];

#### Keyword search and deciding if the the two articles are related/ discuss the same topic

In [11]:
#THIS FUNCTION RETURNS THE 10 MOST FREQUENTLY OCCURING WORDS IN THE GIVEN DOCUMENT.
#IT RETURNS THE WORD AND HOW MANY TIMES IT OCCURS IN THE DOCUMENT.
#IT GIVES A GOOD IDEA ABOUT THE THEME OF THE DOCUMENTS.

def HighFreq(doc):
    L=Vectoriser(doc);
    list=[[i,L[i]] for i in L];
    def MaxFinder(l):
        ind=0;
        max=list[0][1];
        word=l[0][0];
        for i in range(len(l)):
            if l[i][1]>max:
                max=l[i][1];
                ind=i;
                word=l[i][0];
        del l[ind];
        return [word,max];
    b=[];
    for j in range(15):
        if len(list)>0:  
            b+=[MaxFinder(list)];
        else:
            break;
    return b;

In [12]:
#THIS FUNCTION RETURNS THE COMMON WORDS OF THE TWO DOCUMENTS. 
#IT RETURNS ONLY THE HIGH FREQUENCY WORDS.

def CommonWords(doc1,doc2):
    freq1=HighFreq(doc1);
    freq2=HighFreq(doc2);
    common=[];
    for i in freq1:
        if len(freq2)>0:
            for j in range(len(freq2)):
                if i[0]==freq2[j][0]:
                    common+=[i[0]];
                    del freq2[j];
                    break;
    return common;

In [13]:
#THIS FUNCTION IS AN OPTIONAL FUNCTION. IT TAKES AS INPUT A LIST OF KEYWORDS THAT MAY RELATE TO THE DOCUMENTS
#USING WORDNET, THIS FUNCTION EXPANDS THAT LIST TO INCLUDE ALL THE SYNONYMS, OR RELATED WORDS OF THE KEYWORDS.

def related(key):
    newKey=[];
    for i in key:
        syn=wordnet.synsets(i);
        l=[];
        for words in syn:
            l.append(words.lemmas()[0].name());
            w1=words.hyponyms();
            w2=words.part_meronyms();
            w3=words.entailments();
            w=w1+w2+w3;
            for i in w:
                l+=[i.lemmas()[0].name()];
        newKey+=l;
    return newKey;

#### Function to highlight the common words and key words

In [14]:
#THIS FUNCTION IS TO SIMPLIFY THE TASK OF HIGHLIGHTING THE COMMON WORDS. 
#IT HAS NO RELEVANCE TO THE ACTUAL PLAGIARISM DETECTION; IT IS, HOWEVER, NECESSARY FOR THE 
#PURPOSE OF HIGHLIGHTING.

def MergeSort(l):
    n=len(l);
    if n==0 or n==1:
        return l;
    if n==2:
        if l[0][1]>l[1][1]:
            return [l[1],l[0]];
        else:
            return l;
    else:
        n1=n//2;
        a1=l[0:n1];
        a2=l[n1:n];
        a1=MergeSort(a1);
        a2=MergeSort(a2);
        a=[];
        while len(a1)>0 and len(a2)>0:
            if a1[0][1]<=a2[0][1]:
                a+=[a1[0]];
                del a1[0];
            else:
                a+=[a2[0]];
                del a2[0];
        a=a+a1+a2;
        return a;

In [15]:
#THIS FUNCTION COLOURS THE WORDS IN THE DOCUMENT. iT TAKES THE DOCUMENT AS INPUT, ALONG WITH
#THE OUTPUT OF THE MATCHSTRINGS FUNCTION, AND THE FILE WHERE YOU WANT TO WRITE THE OUTPUT. 

def color(f,doc,b,j):
    n=len(doc);
    while j<n and len(b)>0:
        count=0;
        ind=0;
        for i in range(0,len(b)):
            if b[i][1]==j:
                count+=1;
                r=b[i][2]-j;
                if doc[j-1]==" ":
                    if b[i][0]%3==0:
                        for e in range(r):
                            f.write(""'<span style="background-color: green">{}</span>'.format(doc[j+e]));
                    elif b[i][0]%3==1:
                        for e in range(r):
                            f.write(""'<span style="background-color: aqua">{}</span>'.format(doc[j+e]));
                    else:
                        for e in range(r):
                            f.write(""'<span style="background-color: red">{}</span>'.format(doc[j+e]));
                    j+=r;
                    del b[i];
                    break;
                else:
                    for e in range(r):
                        f.write(doc[j+e]);
                    count+=1;
                    j+=r;
                    del b[i];
                    break;
        if count==0:
            f.write(doc[j]);
            j+=1;
    return j;

In [16]:
#tHE ACTUAL FUNCTION TO HIGHLIGHT THE WORDS.
#THE INPUTS N1,N2,N3,N4 REPRESNT THE NAMES OF THE FILES WHERE YOU WANT TO STORE THE HIGHLIGHTED OUTPUT.
#N1 IS FOR COMMON WORDS OF THE FIRST DOCUMENT, N2 IS FOR THE SECOND DOCUMENT.
#THESE VARIABLES SHOULD BE STRINGS, AND BE OF THE FORM 'NAME_OF_FILE.HTML' ONLY.
#N3 AND N4 ARE FOR HIGHLIGHTED KEYWORDS.
#IT TAKES THE 2 DOCUMENTS AS INPUTS, ALONG WITH THE KEYWORD LIST, THE DICTIONARY FOR RABIN-KARP ALGORITHM.

def HighLight(doc1,doc2,key,M,n1,n3):
    n2='temp1.html'
    n4='temp2.html'
    d1=doc1.lower();
    d2=doc2.lower();
    Freq1=HighFreq(d1);
    Freq2=HighFreq(d2);
    Common=[];
    for i in Freq1:
        if len(Freq2)>0:
            for j in range(len(Freq2)):
                if Freq2[j][0]==i[0]:
                    Common+=[i[0]];
                    del Freq2[j];
                    break;
        else:
            break;
                
    KeyWords=related(key);
    b_temp1=[];
    b_temp2=[];
    for i in range(len(Common)):
        b_temp1+=[[i,RabinKarp(d1,Common[i],M),RabinKarp(d2,Common[i],M)]];
    for j in range(len(KeyWords)):
        b_temp2+=[[j,RabinKarp(d1,KeyWords[j],M),RabinKarp(d2,KeyWords[j],M)]];
    CommInd1=[];
    CommInd2=[];
    KeyInd1=[];
    KeyInd2=[];
    for el in b_temp1:
        CommInd1+=[[el[0],i[0],i[1]] for i in el[1]];
        CommInd2+=[[el[0],i[0],i[1]] for i in el[2]];
    for el in b_temp2:
        KeyInd1+=[[el[0],i[0],i[1]] for i in el[1]];
        KeyInd2+=[[el[0],i[0],i[1]] for i in el[2]];
    CommInd1=MergeSort(CommInd1);
    CommInd2=MergeSort(CommInd2);
    KeyInd1=MergeSort(KeyInd1);
    KeyInd2=MergeSort(KeyInd2);
    j1=0;
    j2=0;
    j3=0;
    j4=0;
    f1=open(n1,'w+');
    f2=open(n2,'w+');
    f3=open(n3,'w+');
    f4=open(n4,'w+');
    j1=color(f1,doc1,CommInd1,j1);
    j2=color(f2,doc2,CommInd2,j2);
    j3=color(f3,doc1,KeyInd1,j3);
    j4=color(f4,doc2,KeyInd2,j4);
    a1=len(doc1);
    a2=len(doc2);
    f1.close();
    f2.close();
    f3.close();
    f4.close();
    f1=open(n1,'a');
    f2=open(n2,'a');
    f3=open(n3,'a');
    f4=open(n4,'a');
    f1.write(doc1[j1:a1]);
    f2.write(doc2[j2:a2]);
    f3.write(doc1[j3:a1]);
    f4.write(doc2[j4:a2]);
    f1.close();
    f2.close();
    f3.close();
    f4.close();
    f1=open(n1,'a');
    f2=open(n2,'r');
    f3=open(n3,'a');
    f4=open(n4,'r');
    f1.write("<br><br>")
    f3.write("<br><br>")
    f1.write(f2.read());
    f3.write(f4.read());
    f1.close();
    f2.close();
    f3.close();
    f4.close();

In [26]:
f1=open("text1.txt","r")
f2=open("text2.txt","r")
s1=f1.read()
s2=f2.read()
tkey=""#input("Enter keywords to search for:")
key=fullCleanup(tkey)
print(MatchStrings(s1,s2,Map))
common=CommonWords(s1,s2)
print("The most common words are:",common)
cosine=CosineSimilarity(s1,s2)
print("Cosine similarity: ",cosine)
print("Percentage similarity: ",(math.pi/2-math.acos(cosine))/(math.pi/2)*100,"%")
HighLight(s1,s2,key,Map,'op1.html','op2.html')

[['Number of common sentences', 2], ['Number of sentences in the first document:', 24], ['Number of sentences in the second document:', 22]]
The most common words are: ['people', 'pandemic', 'coronavirus', 'cough', 'infect', 'covid', '2019', 'acute']
Cosine similarity:  0.5868896084988103
Percentage similarity:  39.92954962109474 %


In [25]:
t1="i lost my keys. i like apples."
t2="i misplaced my keys. i love apples."
tkey=input("Enter keywords to search for:")
key=fullCleanup(tkey)
HighLight(t1,t2,key,Map,'op1.html','op3.html')

Enter keywords to search for:lost, like


In [20]:
s1="The legal system is made up of civil courts, criminal courts and specialty courts such as family law courts and bankruptcy court. Each court has its own jurisdiction, which refers to the cases that the court is allowed to hear. In some instances, a case can only be heard in one type of court. For example, a bankruptcy case must be heard in a bankruptcy court. In other instances, there may be several potential courts with jurisdiction. For example, a federal criminal court and a state criminal court would each have jurisdiction over a crime that is a federal drug offense but that is also an offense on the state level."
s2="The legal system is comprised of criminal and civil courts and specialty courts like bankruptcy and family law courts. Every one of the courts is vested with its own jurisdiction. Jurisdiction means the types of cases each court is permitted to rule on. Sometimes, only one type of court can hear a particular case. For instance, bankruptcy cases an be ruled on only in bankruptcy court. In other situations, it is possible for more than one court to have jurisdiction. For instance, both a state and federal criminal court could have authority over a criminal case that is illegal under federal and state drug laws."
common=CommonWords(s1,s2)
print("The most common words are:",common)
cosine=CosineSimilarity(s1,s2)
print("Cosine similarity: ",cosine)
print(math.acos(cosine)*180/math.pi)
print("Percentage similarity: ",(math.pi/2-math.acos(cosine))/(math.pi/2)*100,"%")

The most common words are: ['court', 'criminal', 'bankruptcy', 'jurisdiction', 'case', 'instance', 'federal', 'state', 'legal', 'system']
Cosine similarity:  0.8692279334592989
29.630956023731834
Percentage similarity:  67.07671552918686 %
